In [ ]:
"""Importing the data for building the datasets from """
# # !pip install friends
# import pandas as pd

# train_info = pd.read_csv('./data/wikilarge/grade_ratio_wiki_train.csv')
# val_info = pd.read_csv('./data/wikilarge/grade_ratio_wiki_val.csv')
# test_info = pd.read_csv('./data/wikilarge/grade_ratio_wiki_test.csv')

# with open('./data/wikilarge/wiki_train.src', 'r', encoding='utf-8') as f:
#     train_src = pd.DataFrame(f.readlines(), columns=['src'])
# with open('./data/wikilarge/wiki_val.src', 'r', encoding='utf-8') as f:
#     val_src = pd.DataFrame(f.readlines(), columns=['src'])
# with open('./data/wikilarge/wiki_test.src', 'r', encoding='utf-8') as f:
#     test_src = pd.DataFrame(f.readlines(), columns=['src'])

# with open('./data/wikilarge/wiki_train.tgt', 'r', encoding='utf-8') as f:
#     train_tgt = pd.DataFrame(f.readlines(), columns=['tgt'])
# with open('./data/wikilarge/wiki_val.tgt', 'r', encoding='utf-8') as f:
#     val_tgt = pd.DataFrame(f.readlines(), columns=['tgt'])
# with open('./data/wikilarge/wiki_test.tgt', 'r', encoding='utf-8') as f:
#     test_tgt = pd.DataFrame(f.readlines(), columns=['tgt'])

# train_data = pd.concat([train_info, train_src, train_tgt], axis=1)
# val_data = pd.concat([val_info, val_src, val_tgt], axis=1)
# test_data = pd.concat([test_info, test_src, test_tgt], axis=1)

# all_data = pd.concat([train_data, val_data, test_data], axis=0)
# # subset to src tgt and grade
# all_data = all_data[['src', 'tgt', 'abs_tgt_FKGL_Grade', 'abs_src_FKGL_Grade']]
# # drop duplicates
# all_data = all_data[all_data['src'] != all_data['tgt']]

In [ ]:
"""For building the pickle files for each grade from original CSV files"""
# from sklearn.model_selection import train_test_split
# from collections import defaultdict
# from datasets import DatasetDict, Dataset
# import pickle
# groups = all_data.groupby('abs_tgt_FKGL_Grade')

# group_dict = defaultdict()
# split_dict = defaultdict()
# for group in groups:
#     group_dict[group[0]] = group[1].reset_index(drop=True)
# print(group_dict[0].src[0])

# for group in group_dict.keys():
#     train, test_val = train_test_split(group_dict[group], test_size=1000, random_state=42,)
#     val, test = train_test_split(test_val, test_size=0.5, random_state=42)
#     split_dict[group] = {'train': train.reset_index(drop=True), 'val': val.reset_index(drop=True), 'test': test.reset_index(drop=True)}


# # convert dataframes to datasets and save to pickle format
# for group in split_dict.keys():
#     train_data = Dataset.from_pandas(split_dict[group]['train'])
#     val_data = Dataset.from_pandas(split_dict[group]['val'])
#     test_data = Dataset.from_pandas(split_dict[group]['test'])
#     dataset_dict = DatasetDict({'train': train_data, 'val': val_data, 'test': test_data})

#     with open(f'./data/wikilarge/graded_splits/grade_{group}.pkl', 'wb') as f:
#         pickle.dump(dataset_dict, f)


Though founded in 1887, under Jack Hyles' leadership from 1959 & ndash; 2001 it became one of the megachurches in the United States and during the 1970s had the highest Sunday school attendance of any church in the world.



In [3]:
"""Import pickle, convert back to dataframe, export as json in alpaca format"""
import os
from datasets import DatasetDict
import pickle
import pandas as pd

def reimport_and_save_as_json(grade):
    # Load the pickled dataset

    #system_prompt = "Please rewrite the following sentence to make it easily understandable by students in Grade {tgt_ideal_Grade}. Ensure that the rewritten sentence is grammatically correct, fluent, and retains the core message of the original sentence without changing its meaning."
    instruction_prompt = "Rewrite this Input sentence to make it easily understandable by students in Grade {tgt_ideal_Grade}"# while preserving the meaning: Please note, if the initial rewrite does not meet the specified grade level, you are encouraged to modify and regenerate the output until the criteria are satisfactorily met. The final output should only include the last, correct version of the rewritten sentence(s)."
    
    with open(f'./data/wikilarge/graded_splits_old/picklefiles/grade_{grade}.pkl', 'rb') as file:
        data = pickle.load(file)
    
    dataset = DatasetDict()
    
    for split_str in ["train", "val", "test"]:
        split = pd.DataFrame(data[split_str])
        split['input'] = split['src'].apply(lambda x: x.strip())
        split['output'] = split['tgt'].apply(lambda x: x.strip())

        #split['system'] = split.apply(lambda x: system_prompt.format(tgt_ideal_Grade=grade), axis=1)
        split['instruction'] = split.apply(lambda x: instruction_prompt.format(tgt_ideal_Grade=grade), axis=1)
  
        split['src_grade'] = split['abs_src_FKGL_Grade']
        split['tgt_grade'] = split['abs_tgt_FKGL_Grade']

        split = split[['input', 'output', 'instruction']]#, 'system']]#, 'src_grade', 'tgt_grade']]

        dataset[split_str] = split

    # Save as Alpaca-formatted JSON
    path = f'./data/wikilarge/graded_splits_new/grade_{grade}'
    #os.mkdir(path)
    for split_str in ["train", "val", "test"]:
        
        pd.DataFrame(dataset[split_str]).to_json(f'{path}/{split_str}.json', orient='records', indent=2)

from tqdm import tqdm
# test with grade 8
for grade in tqdm({2,3,4,5,6,7,8,9,10,11,12}):
    reimport_and_save_as_json(grade)


100%|██████████| 11/11 [00:12<00:00,  1.09s/it]
